In [1]:
import pandas as pd
import numpy as np
from statistics import mean, stdev
import matplotlib.pyplot as plt # data visualization
from sklearn.model_selection import StratifiedKFold
import seaborn as sns # statistical data visualization
%matplotlib inline

In [2]:
df = pd.read_csv('/Users/nguyenhalinh/Desktop/Research/dataset.csv')
df = df[df['state_code'] == 'PA']
# df

/var/folders/vf/c31mrqv54zg80c74f5bw2__c0000gn/T/ipykernel_45084/1072782963.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nguyenhalinh/Desktop/Research/csv.csv')


In [3]:
extracted_df = df[df['action_taken'].isin([1,3])] 
extracted_df = extracted_df[extracted_df['loan_purpose'] == 1]
#clean property value column
extracted_df = extracted_df[extracted_df['property_value'].notna()]
extracted_df['property_value'] = np.where(extracted_df['property_value'] == 'Exempt',1111, extracted_df['property_value'])
extracted_df["property_value"] = pd.to_numeric(extracted_df["property_value"])
extracted_df["property_value"] = extracted_df["property_value"]/1000

extracted_df["loan_amount"] = extracted_df["loan_amount"]/1000


# clean loan terms column
extracted_df['loan_term'] = np.where(extracted_df['loan_term'] == 'Exempt',1111, extracted_df['loan_term'])
extracted_df['loan_term'] = pd.to_numeric(extracted_df['loan_term'])

#clean debt to income ratio column
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '<20%',10, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '20%-<30%',25, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '30%-<36%',33, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '50%-60%',55, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '>60%',60, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == 'Exempt',1111, extracted_df['debt_to_income_ratio'])
extracted_df["debt_to_income_ratio"] = pd.to_numeric(extracted_df["debt_to_income_ratio"])

extracted_df = extracted_df[extracted_df['property_value']!= 1111]
extracted_df = extracted_df[extracted_df['debt_to_income_ratio']!= 1111]
extracted_df = extracted_df[extracted_df['loan_term']!= 1111]

extracted_df.loc[extracted_df['action_taken'] == 1, 'outcome'] = 1
extracted_df.loc[extracted_df['action_taken'] == 3, 'outcome'] = 0

extracted_df = extracted_df[extracted_df['debt_to_income_ratio'].notna()]
extracted_df = extracted_df[extracted_df['loan_amount'].notna()]
extracted_df = extracted_df[extracted_df['income'].notna()]
extracted_df = extracted_df[extracted_df['property_value'].notna()]

In [4]:
philly_county = extracted_df[extracted_df['county_code'] == 42101]

In [5]:
philly_county = philly_county[['loan_amount', 'property_value', 'income', 'debt_to_income_ratio','outcome']]

## Data Preparation

In [6]:
X = philly_county.drop(['outcome'], axis=1)

y = philly_county['outcome'].reset_index(drop=True)

In [7]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [8]:
# check the shape of X_train and X_test

# X_train.shape, X_test.shape

In [9]:
# X_train.describe()

In [10]:
cols = X_train.columns
# cols

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [12]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [13]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [14]:
# X_train.describe()

In [15]:
all_X_scaled = scaler.fit_transform(X)

## Model Training

In [16]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression


# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=0)


# fit the model
logreg.fit(X_train, y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [17]:
skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=11)
model = LogisticRegression(solver='liblinear', random_state=0)
lst_accu_stratified=[]

for train_index, test_index in skf.split(X, y): 
    X_train_fold, X_test_fold = all_X_scaled[train_index],all_X_scaled[test_index] 
    y_train_fold, y_test_fold = y[train_index], y[test_index] 
    model.fit(X_train_fold, y_train_fold) 
    lst_accu_stratified.append(model.score(X_test_fold, y_test_fold))
    
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

List of possible accuracy: [0.9049406817840516, 0.9050765995794533, 0.9050765995794533]

Maximum Accuracy That can be obtained from this model is: 90.50765995794534 %

Minimum Accuracy: 90.49406817840516 %

Overall Accuracy: 90.50312936476527 %

Standard Deviation is: 7.847217576281717e-05


## Predicting

In [18]:
y_pred_test = logreg.predict(X_test)

np.unique(y_pred_test)

array([1.])

In [19]:
# probability of getting output as 0 - rejected

logreg.predict_proba(X_test)[:,0]

array([0.05123925, 0.25046292, 0.18363021, ..., 0.06290945, 0.04787176,
       0.14339919])

In [20]:
# probability of getting output as 1 - accepted

logreg.predict_proba(X_test)[:,1]

array([0.94876075, 0.74953708, 0.81636979, ..., 0.93709055, 0.95212824,
       0.85660081])

## Checking accuracy score

In [21]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 0.9071


## Confusion Matrix

In [22]:
# Print the Confusion Matrix and slice it into four pieces

# from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(y_test, y_pred_test)

# print('Confusion matrix\n\n', cm)

# print('\nTrue Positives(TP) = ', cm[0,0])

# print('\nTrue Negatives(TN) = ', cm[1,1])

# print('\nFalse Positives(FP) = ', cm[0,1])

# print('\nFalse Negatives(FN) = ', cm[1,0])

In [23]:
# visualize confusion matrix with seaborn heatmap

# cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                  index=['Predict Positive:1', 'Predict Negative:0'])

# sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

lg = LogisticRegression(solver='liblinear', random_state=0)
lg.fit(X_train,y_train)

y_pred_test = lg.predict(X_test)

pred_proba_df = pd.DataFrame(lg.predict_proba(X_test))
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
    test_accuracy = accuracy_score(y_pred_test,Y_test_pred.iloc[:,1])
    print('Our testing accuracy is {}'.format(test_accuracy))
    print(confusion_matrix(y_pred_test,Y_test_pred.iloc[:,1]))


******** For i = 0.05 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.1 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.15 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.2 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.25 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.3 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.35 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.4 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.45 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.5 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.55 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.6 ******
Our testing accuracy is 1.0
[[7990]]

******** For i = 0.65 ******
Our testing accuracy is 0.9989987484355445
[[   0    0]
 [   8 7982]]

******** For i = 0.7 ******
Our testing accuracy is 0.9868585732165206
[[   0    0]
 [ 105 7885]]

**

## Retest Model

In [26]:
eff_df = pd.read_csv("/Users/nguyenhalinh/Desktop/42101_county_dea.csv")

In [27]:
eff_df = eff_df.drop(eff_df.columns[0],axis=1)
# eff_df = eff_df.loc[eff_df['outcome']==0]
eff_df 

,loan_amount,property_value,income,debt_to_income_ratio,outcome,DEA,eff_loan_amount,eff_property_value,eff_income,eff_debt_to_income_ratio
0,115.0,115.0,51.0,44.0,1.0,0.227273,189.0,210.0,51.0,10.0
1,145.0,145.0,43.0,39.0,1.0,0.256410,188.4,207.6,43.0,10.0
2,165.0,175.0,49.0,25.0,1.0,0.400000,188.8,209.4,49.0,10.0
3,5.0,175.0,49.0,25.0,1.0,0.400000,188.8,209.4,49.0,10.0
4,95.0,115.0,72.0,25.0,1.0,0.400000,190.6,216.1,72.0,10.0
...,...,...,...,...,...,...,...,...,...,...
19970,5.0,85.0,46.0,42.0,1.0,0.238095,188.6,208.5,46.0,10.0
19971,85.0,85.0,46.0,43.0,1.0,0.232558,188.6,208.5,46.0,10.0
19972,525.0,555.0,227.0,25.0,1.0,0.400000,525.0,706.4,589.4,10.0
19973,245.0,305.0,213.0,33.0,1.0,0.303030,393.7,501.4,213.0,10.0


In [28]:
new_test_df = eff_df.drop(eff_df.columns[[0,1,2,3,5]],axis=1)
new_test_df

,outcome,eff_loan_amount,eff_property_value,eff_income,eff_debt_to_income_ratio
0,1.0,189.0,210.0,51.0,10.0
1,1.0,188.4,207.6,43.0,10.0
2,1.0,188.8,209.4,49.0,10.0
3,1.0,188.8,209.4,49.0,10.0
4,1.0,190.6,216.1,72.0,10.0
...,...,...,...,...,...
19970,1.0,188.6,208.5,46.0,10.0
19971,1.0,188.6,208.5,46.0,10.0
19972,1.0,525.0,706.4,589.4,10.0
19973,1.0,393.7,501.4,213.0,10.0


In [29]:
#Rename columns
new_test_df.rename(
    columns={"eff_loan_amount": "loan_amount", "eff_property_value": "property_value", "eff_income": "income", "eff_debt_to_income_ratio":'debt_to_income_ratio'},
    inplace=True,
)
new_test_df

,outcome,loan_amount,property_value,income,debt_to_income_ratio
0,1.0,189.0,210.0,51.0,10.0
1,1.0,188.4,207.6,43.0,10.0
2,1.0,188.8,209.4,49.0,10.0
3,1.0,188.8,209.4,49.0,10.0
4,1.0,190.6,216.1,72.0,10.0
...,...,...,...,...,...
19970,1.0,188.6,208.5,46.0,10.0
19971,1.0,188.6,208.5,46.0,10.0
19972,1.0,525.0,706.4,589.4,10.0
19973,1.0,393.7,501.4,213.0,10.0


In [30]:
#Convert to float data type
new_test_df['loan_amount'] = pd.to_numeric(new_test_df['loan_amount'])
new_test_df['property_value'] = pd.to_numeric(new_test_df['property_value'])
new_test_df['income'] = pd.to_numeric(new_test_df['income'])
new_test_df['debt_to_income_ratio'] = pd.to_numeric(new_test_df['debt_to_income_ratio'])


In [31]:
new_X_test = new_test_df.drop(['outcome'], axis=1)

new_y_test =new_test_df['outcome']

In [32]:
new_X_test.shape

(19975, 4)

In [33]:
cols = new_X_test.columns
cols

Index(['loan_amount', 'property_value', 'income', 'debt_to_income_ratio'], dtype='object')

In [34]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

new_X_test = scaler.transform(new_X_test)

/opt/anaconda3/envs/pyclass/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [35]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [36]:
new_X_test = pd.DataFrame(new_X_test, columns=[cols])

In [37]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train,y_train)

y_pred_test = model.predict(new_X_test)
print(y_pred_test)

pred_proba_df = pd.DataFrame(model.predict_proba(new_X_test))
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
for i in threshold_list:
    print ('\n******** For i = {} ******'.format(i))
    Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
    test_accuracy = accuracy_score(y_pred_test,Y_test_pred.iloc[:,1])
    print('Our testing accuracy is {}'.format(test_accuracy))
    print(confusion_matrix(y_pred_test,Y_test_pred.iloc[:,1]))

[1. 1. 1. ... 1. 1. 1.]

******** For i = 0.05 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.1 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.15 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.2 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.25 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.3 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.35 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.4 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.45 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.5 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.55 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.6 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.65 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.7 ******
Our testing accuracy is 1.0
[[19975]]

******** For i = 0.75 ******
O

In [40]:
all_42011_dea_df = pd.read_csv("/Users/nguyenhalinh/Desktop/Research/all_county_eff_applications.csv")

In [41]:
all_42011_dea_df

,Unnamed: 0,loan_amount,property_value,income,debt_to_income_ratio,outcome,DEA,county_code,weights1,weights2,...,weights12409,weights12410,weights12411,weights12412,weights12413,weights12414,weights12415,weights12416,weights12417,weights12418
0,0,155.0,165.0,115.0,10.0,1.0,1.0,42011,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,155.0,195.0,106.0,10.0,1.0,1.0,42011,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,205.0,255.0,207.0,10.0,1.0,1.0,42011,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,175.0,215.0,92.0,10.0,0.0,1.0,42011,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,65.0,65.0,54.0,10.0,1.0,1.0,42011,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11185,12411,215.0,275.0,187.0,10.0,1.0,1.0,42057,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11186,12412,465.0,665.0,500.0,10.0,1.0,1.0,42057,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11187,12413,265.0,335.0,132.0,10.0,1.0,1.0,42057,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11188,12417,95.0,125.0,107.0,10.0,1.0,1.0,42023,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
